In [1]:
!nvidia-smi

Fri Mar  3 13:45:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.84       Driver Version: 460.84       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0 Off |                  N/A |
| 20%   22C    P8     8W / 250W |   5815MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 20%   

In [2]:
import os
import torch
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, "0" to  "7" 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.cuda.set_device(0)
n_gpu = torch.cuda.device_count()
print(n_gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.get_device_name(0)

1


'GeForce GTX 1080 Ti'

In [3]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm


In [4]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
#Set a seed
import random
import numpy as np
import torch
import datasets
import pandas as pd

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

from torch.utils.data import Dataset, DataLoader

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /DATA/gupta92/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(42)

In [6]:
#Loading dataset

In [7]:
pd.set_option('display.max_colwidth', None)

In [8]:
df = pd.read_csv('DATA/etoori_train.csv')
df.shape
#df = df.head(5000)

(140000, 2)

In [9]:
df.rename(columns = {'enc_input':'input'}, inplace = True)
df.rename(columns = {'dec_input':'output'}, inplace = True)

In [10]:
df.head(10)

,input,output
0,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अप हृदय में गाँठ बनाकर नहीं रखते थे ।,परन्तु वे दोनों उन बातों को ज़्यादा समय तक अपने हृदय में गाँठ बनाकर नहीं रखते थे ।
1,देश में हिन्दी को विस्थापित कर का षड़यंत्र चल रहा है जो चिंता का विषय है ।,देश में हिन्दी को विस्थापित करने का षड़यंत्र चल रहा है जो चिंता का विषय है ।
2,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर लग जा के बाद डेरा प्रेमी इस बार दर्ज हुए मुकदमे में सभी अभियुक्तों की गिरफ्तारी की मांग को लेकर आंदोलनरत हैं ।,तीन साल पहले कातिलाना हमले के प्रकरण में एफआर लग जाने के बाद डेरा प्रेमी इस बार दर्ज हुए मुकदमे में सभी अभियुक्तों की गिरफ्तारी की मांग को लेकर आंदोलनरत हैं ।
3,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम से सिंगापुर के पेरानांका म्यूजियम में प्रदर्शनी लगाई गई हैं ।,रामायण रिविजिटेड अ टेल ऑफ लव एंड एडवेंचर नाम से सिंगापुर के पेरानांका म्यूजियम में प्रदर्शनी लगाई गई है ।
4,तब तक के लिए हमें विराम ले की अनुमति दीजिए ।,तब तक के लिए हमें विराम लेने की अनुमति दीजिए ।
5,देव स्थेन तक सड़क बनाने की परियोजना चल रही है ।,देव स्थान तक सड़क बनाने की परियोजना चल रही है ।
6,वर्ड वाचर व प्रसिद्ध चित्रकार अनूप साह आरएनएस को बताया कि श्वेतकंठ चिलचिल पक्षी का वैज्ञानिक नाम ह्वाइट थ्रोटेट लाफिंग थ्रस है ।,वर्ड वाचर व प्रसिद्ध चित्रकार अनूप साह ने आरएनएस को बताया कि श्वेतकंठ चिलचिल पक्षी का वैज्ञानिक नाम ह्वाइट थ्रोटेट लाफिंग थ्रस है ।
7,दो दिन पहले ज्ञानजी पोस्ट ठेली जिसमें,दो दिन पहले ज्ञानजी ने पोस्ट ठेली जिसमें
8,दुर्घटना की स्थिति में बीमा कम्पनी द्वारा दी जा वाली राशि निर्धारित कर दी गई है ।,दुर्घटना की स्थिति में बीमा कम्पनी द्वारा दी जाने वाली राशि निर्धारित कर दी गई है ।
9,देर से जाऊंगा तो दुकां बंद हो जाएंगी ।,देर से जाऊंगा तो दुकानें बंद हो जाएंगी ।


In [11]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [ ]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#model_name = 't5-base'
# Loaded the trained t5 model (initially trained by pre initializing t5-base) for some epochs 
#(So as to improve the number of epochs for training)
model_name = 't5_gec_hindi'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [14]:
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline
tokenizer = AutoTokenizer.from_pretrained("surajp/RoBERTa-hindi-guj-san",use_fast=False)
#tokenizer = AutoTokenizer.from_pretrained("flax-community/roberta-hindi",use_fast=False)

In [15]:
def calc_token_len(example):
    return len(tokenizer(example).input_ids)

In [16]:
#Train - Test Split
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((126000, 2), (14000, 2))

In [17]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (666 > 512). Running this sequence through the model will result in indexing errors


In [18]:
test_df.head()

,input,output,input_token_len
40665,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है ।,65
48520,तथे गौशाला का,तथा गौशाला का,12
138403,बच्चे खेल रहे है किलकारियाँ गूंज रही है ब्रह्माण्ड में,बच्चे खेल रहे हैं किलकारियाँ गूंज रही हैं ब्रह्माण्ड में,39
130079,धरने पर किसानो ने कहा कि अब समय आ गया हैं कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी हैं ।,धरने पर किसानो ने कहा कि अब समय आ गया है कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी है ।,66
50146,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा हैं ।,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा है ।,57


In [19]:
test_df['input_token_len'].describe()

count    14000.000000
mean        55.171643
std         34.661996
min          6.000000
25%         32.000000
50%         48.000000
75%         70.000000
max        793.000000
Name: input_token_len, dtype: float64

In [20]:
#We will use a token length of 64 since it will cover the vast majority of examples
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [21]:
test_dataset

Dataset({
    features: ['input', 'output', 'input_token_len', '__index_level_0__'],
    num_rows: 14000
})

In [22]:
#Load the Dataset
from torch.utils.data import Dataset, DataLoader
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):         
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 64
  
    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)
    
        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        
        return inputs

  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])
        
        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [23]:
dataset = GrammarDataset(test_dataset, tokenizer, True)
print(dataset[1])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


input_ids 12
attention_mask 12
labels 12
{'input_ids': [0, 2766, 268, 331, 370, 305, 265, 283, 265, 272, 265, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 2766, 265, 331, 370, 305, 265, 283, 265, 272, 265, 2]}


In [24]:
#Define Evaluator
#!pip install rouge_score

In [25]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [26]:
#!pip install evaluate

In [27]:
#Train Model
#data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt')


In [28]:
# defining training related arguments
batch_size = 8

args = Seq2SeqTrainingArguments(
                output_dir="models/hindi/T5_Roberta",
                evaluation_strategy="epoch",
                learning_rate=2e-5,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                weight_decay=0.1,
                save_steps=10000,
                num_train_epochs=2,
                predict_with_generate=True,
                fp16=True,
                )

In [29]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
#os.environ["CUDA_LAUNCH_BLOCKING"] = "false"
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true'

In [30]:
import torch
#torch.cuda.empty_cache()

In [31]:
# defining trainer using 🤗
trainer = Seq2SeqTrainer(model=model, 
                args=args, 
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator)
                #compute_metrics=compute_metrics)

Using cuda_amp half precision backend


In [32]:
#Wandb API Key: bcdbdd5ee9d76a20c90b5f2a246eb45f14b341a9
import os
os.environ["WANDB_DISABLED"] = "true"     #Disabling Wandb
#model.resize_token_embeddings(len(tokenizer))
trainer.train()

***** Running training *****
  Num examples = 126000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 31500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch,Training Loss,Validation Loss
1,0.077000,0.074561
2,0.071400,0.068154


Saving model checkpoint to models/hindi/T5_Roberta/checkpoint-10000
Configuration saved in models/hindi/T5_Roberta/checkpoint-10000/config.json
Model weights saved in models/hindi/T5_Roberta/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in models/hindi/T5_Roberta/checkpoint-10000/tokenizer_config.json
Special tokens file saved in models/hindi/T5_Roberta/checkpoint-10000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 14000
  Batch size = 8
Saving model checkpoint to models/hindi/T5_Roberta/checkpoint-20000
Configuration saved in models/hindi/T5_Roberta/checkpoint-20000/config.json
Model weights saved in models/hindi/T5_Roberta/checkpoint-20000/pytorch_model.bin
tokenizer config file saved in models/hindi/T5_Roberta/checkpoint-20000/tokenizer_config.json
Special tokens file saved in models/hindi/T5_Roberta/checkpoint-20000/special_tokens_map.json
Saving model checkpoint to models/hindi/T5_Roberta/checkpoint-30000
Configuration saved in models/hin

TrainOutput(global_step=31500, training_loss=0.08328946831112816, metrics={'train_runtime': 7967.501, 'train_samples_per_second': 31.628, 'train_steps_per_second': 3.954, 'total_flos': 1.907040123482112e+16, 'train_loss': 0.08328946831112816, 'epoch': 2.0})

In [ ]:
#torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [43]:
#Save the trained Model
trainer.save_model('t5_gec_hindi_Roberta')

Saving model checkpoint to t5_gec_hindi_Roberta
Configuration saved in t5_gec_hindi_Roberta/config.json
Model weights saved in t5_gec_hindi_Roberta/pytorch_model.bin
tokenizer config file saved in t5_gec_hindi_Roberta/tokenizer_config.json
Special tokens file saved in t5_gec_hindi_Roberta/special_tokens_map.json


In [2]:
def correct_grammar(input_text,num_return_sequences):
    batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(device)
    translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [35]:
text = test_df['input'].iat[1]
correct = test_df['output'].iat[1]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: तथे गौशाला का
Correct Seentence: तथा गौशाला का
Predicted Sentence: तथा गौशाला का । 


In [36]:
text = test_df['input'].iat[2000]
correct = test_df['output'].iat[2000]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0][:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: कुछ विश्लेषकों का मत है कि इस कार्यक्रम के कारण लोग निकम्मे हो रहे है ।
Correct Seentence: कुछ विश्लेषकों का मत है कि इस कार्यक्रम के कारण लोग निकम्मे हो रहे हैं । 
Predicted Sentence: कुछ विश्लेषकों का मत है कि इस कार्यक्रम के कारण लोग निकम्मे हो रहे हैं । 


# Testing

In [37]:
#Testing
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,AutoTokenizer
#Loaded the pretrained model
model_name = 't5_gec_hindi_Roberta'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 

tokenizer = AutoTokenizer.from_pretrained("surajp/RoBERTa-hindi-guj-san")
trained_model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

loading configuration file config.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--surajp--RoBERTa-hindi-guj-san/snapshots/4464e6c2a7301ce0b58128b72c679bf6326a4b3b/config.json
Model config RobertaConfig {
  "_name_or_path": "surajp/RoBERTa-hindi-guj-san",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.json from cache at /DATA/gupta92/.cache/huggingface/hub/models--sura

In [5]:
#Testing
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration,AutoTokenizer
model_name = 't5_gec_hindi_Roberta'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu' 
tokenizer = AutoTokenizer.from_pretrained("surajp/RoBERTa-hindi-guj-san")
trained_model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [6]:
trained_model

T5ForConditionalGeneration(
  (shared): Embedding(30522, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(30522, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [7]:
param_size = 0
for param in trained_model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in trained_model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))

model size: 845.604MB


In [8]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

count_parameters(trained_model)

+----------------------------------------------------------------------+------------+
|                               Modules                                | Parameters |
+----------------------------------------------------------------------+------------+
|                            shared.weight                             |  23440896  |
|            encoder.block.0.layer.0.SelfAttention.q.weight            |   589824   |
|            encoder.block.0.layer.0.SelfAttention.k.weight            |   589824   |
|            encoder.block.0.layer.0.SelfAttention.v.weight            |   589824   |
|            encoder.block.0.layer.0.SelfAttention.o.weight            |   589824   |
| encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight |    384     |
|              encoder.block.0.layer.0.layer_norm.weight               |    768     |
|           encoder.block.0.layer.1.DenseReluDense.wi.weight           |  2359296   |
|           encoder.block.0.layer.1.DenseReluDense.wo.

221670144

In [9]:
#model1

In [10]:
def correct_grammar(input_text,num_return_sequences):
    batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
    #translated = model1.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    translated = trained_model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [ ]:
text = test_df['input'].iat[1]
correct = test_df['output'].iat[1]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=4)[0][:l]
print("Predicted Sentence:",predicted_s)

In [42]:
text = test_df['input'].iat[0]
correct = test_df['output'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
l = len(correct)
predicted_s = correct_grammar(text, num_return_sequences=2)[0]
p = len(predicted_s)
#if(p>l):
    #predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।
Correct Seentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है । 
Predicted Sentence: अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थे परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित ह


In [44]:
text = test_df['input'].iat[50]
correct = test_df['output'].iat[50]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: राष्ट्रीय जनता दल हरियाणा आज महिला आरक्षण विधेयक के विरोध में प्रदेश सचिव विजय सिंह कारगवाल के तृत्व में अंबेडकर पार्क के समीप जोदार प्रदर्शन किया तथा कांग्रेस अध्यक्षा सोनिया गांधी व भाजपा की वरिष्ठ त्री सुषमा स्वराज का पुतला फूंका ।
Correct Seentence: राष्ट्रीय जनता दल हरियाणा ने आज महिला आरक्षण विधेयक के विरोध में प्रदेश सचिव विजय सिंह कारगवाल के नेतृत्व में अंबेडकर पार्क के समीप जोदार प्रदर्शन किया तथा कांग्रेस अध्यक्षा सोनिया गांधी व भाजपा की वरिष्ठ नेत्री सुषमा स्वराज का पुतला फूंका । 
Predicted Sentence: राष्ट्रीय जनता दल ने हरियाणा आज महिला आरक्षण विधेयक के विरोध में प्रदेश सचिव विजय सिंह कारगव


In [45]:
text = test_df['input'].iat[60]
correct = test_df['output'].iat[60]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)


Incorrect Sentence: ज़ाहिर है जब मैंने लिखूंगा तो वो मेरे ही व्यक्तिगत विचार कहलायेंगे किसी संस्थान के नहीं ।
Correct Seentence: ज़ाहिर है जब मैं लिखूंगा तो वो मेरे ही व्यक्तिगत विचार कहलायेंगे किसी संस्थान के नहीं । 
Predicted Sentence: ज़ाहिर है जब मैं लिखूंगा तो वो मेरे ही व्यक्तिगत विचार कहलायेंगे किसी संस्थान के नहीं ।


In [46]:
text = test_df['input'].iat[51]
correct = test_df['output'].iat[51]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: गनीमत हैं गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया ।
Correct Seentence: गनीमत है गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप के साये से बचा लिया । 
Predicted Sentence: गनीमत है गणेशशंकर विद्यार्थी ने २०० रुपये का मनीआर्डर भेजकर अशफ़ाक़ की मजार पर छत डलवा कर उसे धूप क


In [48]:
text = test_df['input'].iat[20]
correct = test_df['output'].iat[20]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: हम पास खींच लेते हैंं ।
Correct Seentence: हम पास खींच लेते हैं । 
Predicted Sentence: हम पास खींच लेते हैं । 


# Performance Analysis
1. BLEU Score
2. GLEU Score
3. F0.5 Score

In [49]:
test_df.shape

(14000, 3)

In [50]:
test_df.head(5)

,input,output,input_token_len
40665,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसकी वर्चस्व सर्वत्र स्थापित है ।,अँगरेज़ी सहभाषा के रूप में व्यवहत्त होनी चाहिए थी परंतु आज भी उसका वर्चस्व सर्वत्र स्थापित है ।,65
48520,तथे गौशाला का,तथा गौशाला का,12
138403,बच्चे खेल रहे है किलकारियाँ गूंज रही है ब्रह्माण्ड में,बच्चे खेल रहे हैं किलकारियाँ गूंज रही हैं ब्रह्माण्ड में,39
130079,धरने पर किसानो ने कहा कि अब समय आ गया हैं कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी हैं ।,धरने पर किसानो ने कहा कि अब समय आ गया है कि गन्ने की खेती किसानो के लिये नुकसान का सौदा हो गयी है ।,66
50146,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा हैं ।,सड़क मलबे का ढेर होने से दुपहिया व चार पहिया वाहन चालकों को परेशान होना पड़ रहा है ।,57


In [12]:
import pandas as pd
df_test = pd.read_csv('DATA/etoori_test.csv')
df_test.shape

(30000, 2)

In [13]:
df_test.head(5)

,enc_input,dec_input
0,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरार...
1,यह मन को काबू में करने वाली मुद्रा हैं इसीलिए ...,यह मन को काबू में करने वाली मुद्रा है इसीलिए इ...
2,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ...,आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ प...
3,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...,परिवार के मुताबिक धमाकों में हिस्सा लेने वाले ...
4,उनकी वो वाली बात भी अनिश्चित रहती हैं ।,उनकी वो वाली बात भी अनिश्चित रहती है ।


In [14]:
text = df_test['enc_input'].iat[0]
correct = df_test['dec_input'].iat[0]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]

print("Predicted Sentence:",predicted_s)

Incorrect Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार भी को मिलियन डॉलर क्लब में माना जा रहा है ।
Correct Seentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है । 
Predicted Sentence: इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है ।


In [15]:
%%time
predicted_s = correct_grammar(text, num_return_sequences=1)[0]


CPU times: user 1.27 s, sys: 13.6 ms, total: 1.28 s
Wall time: 1.28 s


In [11]:
predicted_s

'इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है ।'

In [54]:
text = df_test['enc_input'].iat[1]
correct = df_test['dec_input'].iat[1]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: यह मन को काबू में करने वाली मुद्रा हैं इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैंं ।
Correct Seentence: यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं । 
Predicted Sentence: यह मन को काबू में करने वाली मुद्रा है इसीलिए इसे चित्त हस्त मुद्रा योग कहते हैं । 


In [55]:
text = df_test['enc_input'].iat[2]
correct = df_test['dec_input'].iat[2]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा उसनेाँ पर शोर मचा रहा है ।
Correct Seentence: आप पुस्तक पढ़ने में तल्लीन हैं और बच्चा वहाँ पर शोर मचा रहा है । 
Predicted Sentence: आप पुस्तक पढ़ने में तल्लीन है और बच्चा वहाँ पर शोर मचा रहा है ।  


In [56]:
text = df_test['enc_input'].iat[3]
correct = df_test['dec_input'].iat[3]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला हैं ।
Correct Seentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग थे और ये गलत पहचान का मामला है । 
Predicted Sentence: परिवार के मुताबिक धमाकों में हिस्सा लेने वाले कोई दूसरे लोग था और ये गलत पहचान का मामला है ।


In [57]:
text = df_test['enc_input'].iat[4]
correct = df_test['dec_input'].iat[4]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: उनकी वो वाली बात भी अनिश्चित रहती हैं ।
Correct Seentence: उनकी वो वाली बात भी अनिश्चित रहती है । 
Predicted Sentence: उनकी वो वाली बात भी अनिश्चित रहती है । 


In [58]:
text = df_test['enc_input'].iat[9]
correct = df_test['dec_input'].iat[9]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: मैंने ललित हूँ ।
Correct Seentence: मैं ललित हूँ । 
Predicted Sentence: मैं ललित हूँ । 


In [62]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
count=0
test_data = df_test.head(100)
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
    
        #print(act)
        #print(pred_s)
        b = sentence_gleu(act,pred_s)
        if(p<l-4):
            count+=1
        else:
        #print(b)
            GLEU_val_emb.append(b)
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))


(100, 2)


100it [02:08,  1.29s/it]

GELU Score =  0.8957280520863231


In [63]:
print("Sentence with less length:",count)

Sentence with less length: 23


In [86]:
print(df_test.enc_input.str.len().max())
print(df_test.enc_input.str.len().mean())

1849
82.1458


In [76]:
df_test.head(1)

,enc_input,dec_input
0,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार भी को मिलियन डॉलर क्लब में माना जा रहा है ।,इसके अलावा माइकल शूमाकर द्वारा चलाई गई एक फरारी कार को भी मिलियन डॉलर क्लब में माना जा रहा है ।


In [3]:
#Testing if GLEU Score is better for short length sentences or not

In [66]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = df_test.head(10000)
count = 0
index_sentence=[]
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
        
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        if(p<l-4):
            count+=1
            index_sentence.append(ind)
        else:
        #print(b)
            GLEU_val_emb.append(b)
            
        #GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(10000, 2)


1it [00:01,  1.18s/it]

GELU Score for  1  data point 1.0


1001it [22:23,  1.39s/it]

GELU Score for  1001  data point 0.899760730167574


...

GELU Score for  8001  data point 0.9016253094710289


9001it [3:26:10,  1.30s/it]

GELU Score for  9001  data point 0.9010568912145879


10000it [3:48:21,  1.37s/it]

GELU Score =  0.9010645312331664


In [4]:
#Testing GLEU Score for all the senetences in the test dataset

In [17]:
from nltk.translate.gleu_score import sentence_gleu
GLEU_val_emb = []
test_data = df_test.head(10000)
count = 0
index_sentence=[]
print(test_data.shape)
for ind,i in tqdm(test_data.iterrows(),position=0):
    try:
        text = str(i.enc_input)
        #pred = predict(str(i.enc_input),model)[0].split()
        pred = correct_grammar(text, num_return_sequences=1)[0]
        l = len(text)
        p = len(pred)
        if(p>l):
            pred = pred[:l]
        
            
        act = [str(i.dec_input).split()]
        pred_s = str(pred).split()
        b = sentence_gleu(act,pred_s)
        '''
        if(p<l-4):
            count+=1
            index_sentence.append(ind)
        else:
        #print(b)
            GLEU_val_emb.append(b)
        '''
            
        GLEU_val_emb.append(b)
        if(ind%1000 ==0):
            print("GELU Score for ",ind+1," data point",np.mean(GLEU_val_emb))
    except:
        continue
print("GELU Score = ",np.mean(GLEU_val_emb))

(10000, 2)


1it [00:01,  1.20s/it]

GELU Score for  1  data point 1.0


1001it [19:33,  1.17s/it]

GELU Score for  1001  data point 0.8216028998421867


...

GELU Score for  8001  data point 0.818803948107944


9001it [2:45:30,  1.02s/it]

GELU Score for  9001  data point 0.8190208595040448


10000it [3:03:04,  1.10s/it]

GELU Score =  0.8201520843345623


In [72]:
text = df_test['enc_input'].iat[173]
correct = df_test['dec_input'].iat[173]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l+1]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: इस दिन बुक फेयर में आ वाले बच्चों को रिफ्रेशमेंट भी मिलेगी ।
Correct Seentence: इस दिन बुक फेयर में आने वाले बच्चों को रिफ्रेशमेंट भी मिलेगी । 
Predicted Sentence: इस दिन बुक फेयर में आने वाले बच्चों को रिफ्रेशमेंटने भी मिलेगी ।


In [73]:
text = df_test['enc_input'].iat[113]
correct = df_test['dec_input'].iat[113]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l+1]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: महानगर में बन वाले फ्लाई ओवर्स और आरओबी में अब साइकिल ट्रैक बनाना अनिवार्य होगा ।
Correct Seentence: महानगर में बनने वाले फ्लाई ओवर्स और आरओबी में अब साइकिल ट्रैक बनाना अनिवार्य होगा । 
Predicted Sentence: महानगर में बनने वाले फ्लाई ओवर्स और आरओबी में अब साइकिल ट्रैक बनाना अनिवार्य होगा । 


In [75]:
text = df_test['enc_input'].iat[185]
correct = df_test['dec_input'].iat[185]
print("Incorrect Sentence:", text)
print("Correct Seentence:",correct)
predicted_s = correct_grammar(text, num_return_sequences=1)[0]
l = len(correct)
p = len(predicted_s)
if(p>l):
    predicted_s = predicted_s[:l+1]
print("Predicted Sentence:",predicted_s)

Incorrect Sentence: नानाजी देशमुख तो उनके दाएं हाथ थी ।
Correct Seentence: नानाजी देशमुख तो उनके दाएं हाथ थे । 
Predicted Sentence: नानाजी देशमुख तो उनके दाएं हाथ थे ।  


In [5]:
#Checking the index of sentences which are giving good results

In [67]:
print(count)
print(index_sentence)

2870
[10, 11, 12, 13, 14, 16, 21, 40, 41, 49, 57, 58, 59, 60, 61, 65, 68, 69, 76, 78, 80, 89, 97, 106, 107, 108, 109, 112, 116, 118, 122, 124, 127, 128, 137, 138, 140, 146, 150, 151, 156, 157, 167, 168, 171, 174, 176, 177, 183, 186, 187, 188, 189, 190, 194, 195, 197, 198, 199, 208, 217, 232, 236, 246, 248, 250, 255, 256, 262, 264, 266, 268, 272, 273, 275, 283, 291, 292, 294, 296, 298, 299, 305, 309, 311, 315, 316, 318, 326, 337, 341, 342, 345, 348, 350, 352, 355, 356, 357, 358, 359, 365, 366, 368, 378, 381, 389, 390, 395, 398, 400, 401, 402, 410, 413, 414, 415, 431, 437, 445, 446, 451, 455, 458, 461, 462, 463, 465, 469, 470, 472, 476, 480, 483, 484, 485, 492, 499, 500, 501, 504, 510, 513, 514, 518, 520, 526, 531, 542, 543, 550, 557, 559, 561, 563, 564, 565, 566, 574, 594, 596, 597, 600, 601, 602, 603, 610, 612, 613, 614, 616, 621, 622, 631, 632, 635, 637, 638, 640, 645, 646, 647, 649, 655, 656, 657, 658, 659, 660, 661, 662, 666, 668, 670, 671, 679, 684, 685, 691, 692, 711, 712, 713, 71